In [1]:
import os
os.chdir('../')

In [2]:
%matplotlib inline
#%matplotlib notebook

%load_ext autoreload
%autoreload 2

In [3]:
from copy import deepcopy
from decimal import Decimal
from typing import List, Tuple
from warnings import warn

from cycler import cycler
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.decomposition import PCA
import scipy.stats as stats
import torch
from torch import nn
import torch.nn.functional as F
import seml
from ogb.nodeproppred import PygNodePropPredDataset

from torch_geometric.utils import subgraph
from rgnn_at_scale.utils import accuracy


import tqdm
tqdm.tqdm.pandas()

/nfs/staff-ssd/geisler/anaconda3/envs/rgnn_at_scale/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
from notebooks import mpl_latex

/nfs/staff-ssd/geisler/anaconda3/envs/rgnn_at_scale/lib/python3.8/_collections_abc.py:832: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


In [5]:
mpl_latex.enable_production_mode()

In [6]:
df_experiments = seml.get_results('kdd21_rgnn_at_scale_attack_evasion_transfer_vs_number_nodes',
                                  to_data_frame=True,
                                  fields=['batch_id', 'slurm', 'config', 'result'])

df_experiments = df_experiments[df_experiments['config.use_largest_component'].fillna(False)]

df_experiments.head()

RuntimeError: CUDA error: out of memory

In [ ]:
df_experiments.columns

In [ ]:
df_results = [
    pd.DataFrame(r) 
    for r in df_experiments['result.results']
]
for df_result, (_, df_experiment) in zip(df_results, df_experiments.iterrows()):
    df_result['dataset'] = df_experiment['config.dataset']
    df_result['attack'] = df_experiment['config.attack']
    df_result['block_size'] = df_experiment['config.attack_params.search_space_size']
    df_result['seed'] = df_experiment['config.seed']
    df_result['novel_loss'] = (
        (df_experiment['config.attack_params.loss_type'] == 'MCE')
        | (df_experiment['config.attack_params.loss_type'] == 'tanhCW')
    )


df_results = pd.concat(df_results, ignore_index=True)
df_results

In [ ]:
for epsilon in np.unique(df_results.epsilon):
    df = df_results[df_results.epsilon == epsilon]
    fig, ax = mpl_latex.newfig(width=0.30, ratio_yx=0.6)
    #plt.scatter(df.n_nodes, df.accuracy)
    sns.regplot(df.n_nodes, df.accuracy)
    plt.xlabel('Number of nodes')
    plt.ylabel('Accuracy')
    correlation, p_value = stats.pearsonr(df.n_nodes, df.accuracy)
    plt.title(f'Epsion={epsilon}, correlation={correlation}, p-value={p_value}')
    mpl_latex.savefig(f'latex/assets/global_prbcd_arxiv_{epsilon}_graph_size_vs_robustness_no_legend', fig, close_fig=False)
    plt.legend()
    mpl_latex.savefig(f'latex/assets/global_prbcd_arxiv_{epsilon}_graph_size_vs_robustness', fig, close_fig=False)
    mpl_latex.dedicated_legend_plot(
        f'latex/assets/global_prbcd_arxiv_{epsilon}_graph_size_vs_robustness_legend',
        mod_label=lambda l: l,
        ncol=3
    )

In [ ]:
for epsilon in np.unique(df_results.epsilon):
    df = df_results[df_results.epsilon == epsilon]
    df = df.merge(df_results[df_results.epsilon == 0], on='seed', suffixes=('', '_zero_eps'))
    #df = df[(df.accuracy_zero_eps - df.accuracy) <= 0.15]
    fig, ax = mpl_latex.newfig(width=0.30, ratio_yx=1)
    #plt.scatter(df.n_nodes, df.accuracy)
    sns.regplot(df.n_nodes, df.accuracy_zero_eps - df.accuracy)
    plt.xlabel('Number of nodes')
    plt.ylabel('Accuracy drop')
    correlation, p_value = stats.pearsonr(df.n_nodes, df.accuracy_zero_eps - df.accuracy)
    plt.title(f'Epsion={epsilon}, correlation={correlation}, p-value={p_value}')
    mpl_latex.savefig(f'latex/assets/global_prbcd_arxiv_{epsilon}_graph_size_vs_robustness_drop_no_legend', fig, close_fig=False)
    plt.legend()
    mpl_latex.savefig(f'latex/assets/global_prbcd_arxiv_{epsilon}_graph_size_vs_robustness_drop', fig, close_fig=False)
    mpl_latex.dedicated_legend_plot(
        f'latex/assets/global_prbcd_arxiv_{epsilon}_graph_size_vs_robustness_drop_legend',
        mod_label=lambda l: l,
        ncol=3
    )

In [ ]:
'q'/2

import statsmodels.api as sm
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

for epsilon in np.unique(df_results.epsilon):
    df = df_results[df_results.epsilon == epsilon]
    df = df.merge(df_results[df_results.epsilon == 0], on='seed', suffixes=('', '_zero_eps'))
    fig, ax = mpl_latex.newfig(width=0.30, ratio_yx=0.6)
    results = smf.ols('accuracy_zero_eps ~ n_nodes', data=df).fit()
    sm.graphics.influence_plot(results)
    plt.xlabel('Number of nodes')
    plt.ylabel('Accuracy drop')
    plt.title(f'Epsion={epsilon}, correlation={correlation}, p-value={p_value}')

    
